## Hausaufgabe 2

#### **Abgabe** <br> 29.04. 12:00 Uhr (via studIP) 
#### **Verteidigung** <br> 02.05. 08:00-10:00 Uhr 

Mit jedem Blatt können **maximal** 5 Prozentpunkte Bonus für die Klausur erreicht werden, falls:
1) mind. 60% des Arbeitsblattes korrekt bearbeitet wurden 
2) das Arbeitsblatt erfolgreich verteidigt (den Tutoren erklärt) wurde.

*Rahmenbedingungen*:
- Jede Gruppe muss sich bei studIP für einen Slot zur Verteidigung registrieren.
- Erfolgreiche Verteidigung bedeutet, dass ein von uns ausgewähltes Team-Mitglied die Lösung erklärt und Fragen dazu beantwortet.
- ChatGPT und andere Hilfsmittel dürfen verwendet werden, müssen aber zitiert werden.
- Der Code muss ohne Hilfsmittel erläutert werden.
- Wir bewerten nicht nur die Korrektheit der Ergebnisse, sondern auch ob der Code effizient, einfach zu verstehen, aufgeräumt und gut dokumentiert ist.
- Jede Gruppe gibt nur eine Lösung als Zip-Datei ab, die mit dem Namen und der Matrikelnummer eines Mitglieds versehen ist. Also z.B. `mustermann_12345678.zip` 

**ACHTUNG: Nutzen Sie zum Lösen der Aufgaben ausschließlich Bash-Befehle!**

Installatieren Sie (falls nicht vorhanden) den bash-kernel durch: `pip install bash_kernel ; python -m bash_kernel.install`

_________

In dieser Aufgabe ist das Ziel, durch einzelne kurze Skripts einen Text zu bearbeiten, sodass dieser anschließend einfacher zu verwenden ist (z.B. für Data-Science Anwendungen)

Der Text den wir im Folgenden bearbeiten werden ist das Deutsche Grundgesetz. 

## 1 a)  [2,5%]

Erstellen Sie einen neuen Ordner und laden Sie den Text herunter den Sie unter folgender Adresse finden: https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt 

In [28]:
mkdir DeutscheGrundgesetz ; cd DeutscheGrundgesetz ; wget https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt 

--2024-04-26 20:27:22--  https://raw.githubusercontent.com/levinalex/deutsche_verfassungen/master/grundgesetz/grundgesetz.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...


Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146482 (143K) [text/plain]
Saving to: ‘grundgesetz.txt’

grundgesetz.txt     100%[===================>] 143.05K  --.-KB/s    in 0.04s   

2024-04-26 20:27:22 (3.38 MB/s) - ‘grundgesetz.txt’ saved [146482/146482]



## 1 b) [10%]
Der Text hat eine klare Struktur. Aber am Anfang findet sich ein Stück Text welches nicht in diese Struktur passt. Schreiben Sie ein einzeiliges Skript welches alles vor dem Start der Auflistung der Artikel (und Kapitel) aus dem Text löscht. Speichern Sie die Ausgabe in einer neuen Datei. 
Stellen Sie sicher dass Ihr Skript auch funktioniert wenn sich der Text am Start ändert!

In [29]:
sed -n '/^I$/,$p' grundgesetz.txt > überarbeitetes_grundgesetz.txt

# -n = unterdrückt ausgabe während des befehls
# ^I$ = bis zur Zeile in der nur das I vorkommt wird alles gelöscht
# $p = das I in der gefundenen Zeile soll aber noch gedruckt werden

## 1 c) [10%]
Splitten Sie den Text in einzelne Dateien auf basierend auf den Kapiteln (z.B. "I Die Grundrechte") nummerieren Sie die Dateien.

In [63]:
awk '{
    if (match($0, /[IVXLCDM]+[a-z]?$/) && !match($0, /[a-z]{2}/) && NF == 1) {         #wenn die Zeile mit einer römischen Ziffer, optional gefolgt von einem Kleinbuchstaben, beginnt und nur ein Element enthält wird neue Datei angelegt
        filename = "Kapitel_" $1 ".txt"                                                #Datei wird mit römischer Zahl benannt
        print > filename
    } else {                                                                           #folgende Zeilen werden in neu erstellte Datei angefügt
        print >> filename
    }
}' überarbeitetes_grundgesetz.txt

## 1 d) [30%]
Splitten Sie jedes Kapitel in die einzelnen Artikel auf. Jeder Artikel soll in einer eigenen Datei sein welche die Nummer des Artikels als Dateinamen. Erstellen Sie für jedes Kapitel einen neuen Unterordner in dem die Artikel gesammelt werden.

In [15]:
find . -type f  -name "*Kapitel_*.txt"  -exec sh -c '       #find Befehl soll nur nach Dateien suchen                            
                                                            #die gesuchten Dateien müssen "Kapitel_" im Namen haben
                                                            #für jede gefundene Datei wird folgender command in '' ausgeführt
        dir="${1%.*}"                                       #Variable dir mit Namen der Datei (also Kapitel_x) ohne Anhang .txt
        mkdir -p "$dir"                                     #Kapitel-Unterordner wird angelegt
        awk -v dir="$dir" "                                 #Inhalt von Kapitel_x.txt wird zeilenweise gelesen; -v bewahrt die dir Variable im awk Befehl 
            {
                if (\$1 ~ /^Artikel/ && NF < 4) {           #Wenn eine Zeile mit "Artikel" anfängt und sie weniger als 4 Elemente beinhaltet wird eine .txt Datei im Unterordner mit dem Namen des Artikels erstellt
                    filename = dir \"/\" \$0 \".txt\"
                    print > filename
                } else if (filename != \"\") {              #Ansonsten werden die übrigen Zeilen in die zuletzt erstellte Datei angefügt; 
                    print >> filename                           #aber nur wenn auch eine Datei vorhanden ist (umgehen von Kapitelüberschrift)
                }
            }
        " "$1"
    ' _ {} \;

## 1 e) [5%]
Löschen Sie die ursprünglichen Daten und die Daten welche Sie im Verlauf des Prozesses (abgesehen von 1d)) generiert haben. Am Ende sollen nur noch die Dateien zu den einzelnen Artikel (in getrennten Ordnern) vorhanden sein.  

In [8]:
rm -f ./*

#löscht alle Dateien aber keine Verzeichnisse und die Dateien in ihnen

rm: cannot remove './Kapitel_I': Is a directory
rm: cannot remove './Kapitel_II': Is a directory
rm: cannot remove './Kapitel_III': Is a directory
rm: cannot remove './Kapitel_IV': Is a directory
rm: cannot remove './Kapitel_IVa': Is a directory
rm: cannot remove './Kapitel_IX': Is a directory
rm: cannot remove './Kapitel_V': Is a directory
rm: cannot remove './Kapitel_VI': Is a directory
rm: cannot remove './Kapitel_VII': Is a directory
rm: cannot remove './Kapitel_X': Is a directory
rm: cannot remove './Kapitel_Xa': Is a directory
rm: cannot remove './Kapitel_XI': Is a directory


: 1

## 1 f) [2,5%]
Packen Sie den Inhalt des gesamten Ordners in komprimiertes tar-Archiv.

In [9]:
cd .. && tar -czf deutscheGrundgesetz.tar.gz DeutscheGrundgesetz

_________

## 2 a) [20%]
Schreiben Sie ein Shell-Skript, dass alle x Sekunden ausgibt ob ein durch eine PID identifizierter Prozess läuft oder nicht.
Übergeben Sie die Anzahl an Sekunden und die PID als Parameter an das Skript, sodass es wie folgt aufgerufen werden kann:

```bash
$ script [PID] [SECONDS]
```
Ihnen sind keine Einschränkungen gegeben, welche Befehle Sie dafür verwenden.

**Nützliche Befehle:** `grep`,`ps`

### <span style="color:green">Erklärungen</span>

PID = Ein Process identifier (process ID, PID; deutsch Prozesskennung) ist in der Informatik ein einzigartiger Schlüssel, welcher der eindeutigen Identifikation von Prozessen dient. Der PID ändert sich während der Laufzeit des Prozesses nicht.

In [ ]:
#Shell-SKript erstellen und ausführ-Rechte erteilen

touch pid_check.sh && chmod +x pid_check.sh

In [ ]:
##Inhalt des Skripts "pid_check.sh" (zur Erinnerung; nicht ausführen!)

#!/bin/bash

# Die ersten beiden Argumente in Variablen speichern
pid="$1"
seconds="$2"

#counter zum zählen der insg vergangenen Sekunden
counter=0

#Schleife die die Überprüfung alle x sekunden ausführt
while true; do 
    if ps -p $pid > /dev/null; then
        echo "Der Prozess, identifiziert durch die PID $pid, läuft."
    else 
        echo "Der Prozess, identifiziert durch die PID $pid, läuft nicht."
    fi
    sleep $seconds

    #Endlosschleifen abbruch nach ca 20 sekunden
    ((counter+=$seconds))
    if ((counter >= 20)); then
        echo "Die Überprüfung wurde nach $counter Sekunden beendet."
        break
    fi

done

In [10]:
#Shell-Skript ausführen

./pid_check.sh 1234 5

Der Prozess, identifiziert durch die PID 1234, läuft nicht.
Der Prozess, identifiziert durch die PID 1234, läuft nicht.
Der Prozess, identifiziert durch die PID 1234, läuft nicht.
Der Prozess, identifiziert durch die PID 1234, läuft nicht.
Die Überprüfung wurde nach 20 Sekunden beendet.


## 2 b) [20%]
Schreiben Sie ein Shell-Skript, das die Verzeichnisstruktur eines als Parameter angegebenen Verzeichnisses auflisten kann. Schreiben Sie dazu eine rekursive Funktion, die in Unterverzeichnisse herabsteigt. 
Geben Sie das Verzeichnis und die Dateien auf dem Bildschirm aus. 
Nutzen Sie Einrückungen um die Zugehörigkeit von Dateien und Verzeichnissen zueinander zu kennzeichnen. 
Dies kann dann zum Beispiel so aussehen:

```
File: linux-3.14/COPYING
File: linux-3.14/CREDITS
Directory: linux-3.14/Documentation
    File: linux-3.14/Documentation/00-INDEX 
    Directory: linux-3.14/Documentation/ABI
        File: linux-3.14/Documentation/ABI/README 
        Directory: linux-3.14/Documentation/ABI/obsolete        
            File: linux-3.14/Documentation/ABI/obsolete/proc-sys-vm-nr_pdflush_threads 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-bus-usb
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-class-rfkill
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-koneplus 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-kovaplus 
            File: linux-3.14/Documentation/ABI/obsolete/sysfs-driver-hid-roccat-pyra
        Directory: linux-3.14/Documentation/ABI/removed 
            File: linux-3.14/Documentation/ABI/removed/devfs 
            File: linux-3.14/Documentation/ABI/removed/dv1394
...

```

**Hinweis:** es ist möglich, in der bash Funktionen zu definieren. Die Syntax finden Sie z.B. hier: http://tldp.org/LDP/abs/html/functions.html

**Nützliche Befehle:** `find`,`printf`

In [ ]:
#Shell-SKript erstellen und ausführ-Rechte erteilen

touch verzeichnisstruktur.sh && chmod +x verzeichnisstruktur.sh

In [ ]:
##Inhalt des Skripts "verzeichnisstruktur.sh" (zur Erinnerung; nicht ausführen!)

#!/bin/bash

#Funktion zum Verzeichnis-ausgeben
dir_content() {
   
    #lokale Variablen zur klaren Trennung innerhal und außerhalb der Funktion
    local dir="$1"      #Directory
    local tab="$2"      #Einrückungs-platzhalter

    #aktuelles Verzeichnis wird ausgegeben
    echo "${tab}Directory: $dir"
    local tab="$2    "  #Einrückung muss für nächste Stufe erweitert werden

    #alle Elemente im Verzeichnis werden durchlaufen
    for item in "$dir"/*; do
        if [ -d "$item" ]; then                 #-d prüft ob das Element ein Verzeichnis ist
            dir_content "$item" "$tab    "      #rekursiver Aufruf um die Elemente im gefundenen Verzeichnis aufzulisten
        else
            echo "${tab}File: ${item}"          #Schöner:? aktuelle Datei wird ausgegeben; "##*/" bestimmt, dass Dateinamen ohne Directory ausgegeben wird (also ohne alles bis zum letzten /)
        fi
    done

}

#Funktion zum Verzeichnis-ausgeben wird mit Verzeichnis als Parameter aufgerufen
dir_content "$1" ""

In [10]:
#Shell-Skript ausführen

./verzeichnisstruktur.sh DeutscheGrundgesetz

Directory: DeutscheGrundgesetz
    File: deutscheGrundgesetz.tar.gz
        Directory: DeutscheGrundgesetz/Kapitel_I
            File: Artikel 10.txt
            File: Artikel 11.txt
            File: Artikel 12 a.txt
            File: Artikel 12.txt
            File: Artikel 13.txt
            File: Artikel 14.txt
            File: Artikel 15.txt
            File: Artikel 16 a.txt
            File: Artikel 16.txt
            File: Artikel 17 a.txt
            File: Artikel 17.txt
            File: Artikel 18.txt
            File: Artikel 19.txt
            File: Artikel 1.txt
            File: Artikel 2.txt
            File: Artikel 3.txt
            File: Artikel 4.txt
            File: Artikel 5.txt
            File: Artikel 6.txt
            File: Artikel 7.txt
            File: Artikel 8.txt
            File: Artikel 9.txt
        Directory: DeutscheGrundgesetz/Kapitel_II
            File: Artikel 20 a.txt
            File: Artikel 20.txt
            File: Artikel 21.txt
         